[View in Colaboratory](https://colab.research.google.com/github/irahel/Algorithms-In-Graphs/blob/master/DFS_TopologicalOrdering.ipynb)

In [0]:
# Authenticate and link with Google Drive

from google.colab import auth
auth.authenticate_user()

!pip install -U -q PyDrive
from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
from oauth2client.client import GoogleCredentials

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

myfile = drive.CreateFile({'id': '1pyBQLBZsV2-1q2NWoyS5sirEAECk9aOW'})
myfile.GetContentFile('toy_cycle.rud')

In [0]:
# Object Node
#
# Params:
#
# father:   Node Father
# label:    Node identification
# color:    Node color can be white, black or gray
# started:  Node time started
# finished: Node time finished
# edges:    Node number of edges, one edge can be (Árvore, Retorno, Avanço e Cruzamento)

class Node:
    def __init__(self, father = None, label = -1, color = 'white', edges = 0, started = 0, finished = 0):
        self.father = father
        self.label = label
        self.color = color
        self.started = started
        self.finished = finished
        self.edges = edges

In [0]:
# Object Tree
#
# Params:
#
# list_of_nodes: List of nodes in tree
#
# Methods:
#
# insert_node:
#     Require:
#        node: Node to insert into the tree
#     Result: 
#        Insert a node in @list_of_nodes

class Tree:
  def __init__(self):
    self.list_of_nodes = []
    
  def insert_node(self, node):
    self.list_of_nodes.append(node)        

In [0]:
# Object Graph
#
# Params:
#
# instance:             .txt graph instance to work
# lines:                Lines of txt
# reverse:              Whether the search will be done in reverse order or not
# cnt_vertex:           Number of vertices of the graph
# cnt_edges:            Number of edges of the graph
# adjacency_list:       List of adjacencies of vertices in the graph
# list_of_nodes:        List of nodes in the graph
# dict_of_edges:        Maps as edges with their proper classification (Árvore, Retorno, Avanço e Cruzamento)
# list_of_trees:        List of created trees
# topological_ordering: Vector with topological ordering of vertices
# cnt_trees:            Number of created trees
# time:                 Atual step count
#
# Methods:
#
# list_type_of_edges:
#     Require:
#         Nothing
#     Result:
#         Shows all the edges present in the graph and their respective classification
#
# expand_node:
#     Require:
#         node: Node to expand
#     Result:
#         'Expand' a @node, marks the nodes as gray or black and adds in the stack the white nodes
#
# show_nodes:
#     Require:
#         Nothing
#     Result:
#         Shows all the nodes in the graph, its start and end time
#
# show_topological:
#     Require:
#         Nothing
#     Result:
#         Shows the topological ordering of the graph
#
# verify:
#     Require:
#         node: Node to verify
#     Result:
#         Verifies that the node can be 'closed', ie its children are all 'black'
#
# dfs_search:
#     Require:
#         Nothing
#     Result:
#         Performs an depth firts search in the graph
#
# show_trees:
#     Require:
#         Nothing
#     Result:
#         Shows all trees created
#
# reverse_graph:
#     Require:
#         Nothing
#     Result:
#         Inverts the current graph

class Graph:
    def __init__(self, instance_name):
        # Load instance
        instance = open(instance_name)
        lines = instance.readlines()
        self.reverse = False
        self.cnt_vertex, self.cnt_edges = [int(x) for x in lines[0].split()]     
        
        ####### Initialize all variabels ########
        self.adjacency_list = [[] for x in range(self.cnt_vertex)]
        self.list_of_nodes = [Node(label = x) for x in range(self.cnt_vertex)]
        self.dict_of_edges = dict()
        self.list_of_trees = [Tree()]
        self.topological_ordering = []
        self.cnt_trees = 0
        self.time = 0
        self.reverse = False
        #########################################
        
        for l in range(self.cnt_edges):
            v1, v2, value = [int(x) for x in lines[l+1].split()]
            self.adjacency_list[v1-1].append(v2-1)
            self.list_of_nodes[v1-1].edges += 1
            self.list_of_nodes[v2-1].edges += 1
            self.dict_of_edges[(v1-1, v2-1)] = None
      # End
     
    def list_type_of_edges(self):
      # Need to set only (Cruzamento e Avanço)
      for u, v in self.dict_of_edges.keys():
        edge = self.dict_of_edges[(u, v)]
        
        crossover = True
      
        for tree in self.list_of_trees:
          nodes_in_tree = [tree.list_of_nodes[x].label for x in range(len(tree.list_of_nodes))]
          
          if (u in nodes_in_tree and v in nodes_in_tree):
            if tree.list_of_nodes[v].father != None and tree.list_of_nodes[v].father.label != u:
              self.dict_of_edges[(u, v)] = 'Avanço'
              crossover = False
              break
          
          if edge == None and crossover:
            self.dict_of_edges[(u, v)] = 'Cruzamento'
      
      # Print   
      for u, v in self.dict_of_edges.keys():
        edge = self.dict_of_edges[(u, v)]
        print(str(u) + ' ' + str(v) + ' ' + str(edge))
   
    def expand_node(self, node):
        self.time += 1
        node.color = 'grey'
        node_is_black = True       
        node.started = self.time
        self.list_of_trees[self.cnt_trees].insert_node(node)
        
        # Catch the 'white' nodes and append them on the stack
        # and set edge classification (Arvore e Retorno)
        self.adjacency_list[node.label] = sorted(self.adjacency_list[node.label], reverse=True)
        
        for label_node in self.adjacency_list[node.label]:
            node_expanded = self.list_of_nodes[label_node]
            if node_expanded.color == 'white':
                if node_expanded in self.expand_stack:
                  self.expand_stack.remove(node_expanded)

                self.dict_of_edges[(node.label, node_expanded.label)] = 'Arvore'
               
                self.expand_stack.append(node_expanded)
                node_expanded.father = node
                node_is_black = False
                
            if node_expanded.color == 'grey':
              self.dict_of_edges[(node.label, node_expanded.label)] = 'Retorno'
        
        # If the expanded-node has no 'white' child, then we finish that node
        if node_is_black:
            node.color = 'black'
            self.time += 1
            node.finished = self.time
            
            self.topological_ordering.append(node)
            
            return node.father
    
    def show_nodes(self):
        for node in self.list_of_nodes:
            print('Node: ' + str(node.label) + ' Started at: ' + str(node.started) + ' and Finished at: ' 
                + str(node.finished) + '\n')         
            
    def show_topological(self):
      list_top = self.topological_ordering.copy()
      
      to_print = ""
      for index in range(len(list_top)):
        node = list_top.pop()
        to_print += 'Node' +str(node.label) + ' --> '
      print(to_print)
    
    # Used in dfs
    def verify(self, node):
      can_close = True
      
      for adjacency_label in self.adjacency_list[node.label]:
        child = self.list_of_nodes[adjacency_label]
        if not child.color == 'black':
          can_close = False
          break  
    
      if can_close:
        node.color = 'black'
        self.time += 1
        node.finished = self.time
        
        self.topological_ordering.append(node)
        
        return node.father
    
    # Depth First Search
    def dfs_search(self):
        if self.reverse:
          self.expand_stack = [self.topological_ordering.pop()]
        else:
          self.expand_stack = [max(self.list_of_nodes, key=lambda n : n.edges)]
        
        while True:            
          while len(self.expand_stack) > 0:
              node = self.expand_node(self.expand_stack.pop())
              
              if node != None: 
                father = self.verify(node)
                
                while father != None:
                  father = self.verify(father)
                
          
          has_white = False
          # Verifies if exists white nodes in graph
          if len(self.expand_stack) == 0:
            if self.reverse:
              list_top = self.topological_ordering.copy()
              
              for i in range(len(list_top)):
                node = list_top.pop()
                if node.color == 'white':
                  self.expand_stack.append(node)
                  has_white = True
                  break
            else:      
              for node in self.list_of_nodes:
                if node.color == 'white':
                  self.expand_stack.append(node)
                  has_white = True
                  break                
                
          if not has_white:
            break
          else:
            self.list_of_trees.append(Tree())
            self.cnt_trees += 1
            
    def show_trees(self):
      #for t in self.list_of_trees:
        print('####################################################')
        
        for index in range(len(self.list_of_trees)):
          result = ''
          for node in self.list_of_trees[index].list_of_nodes:
            result += str(node.label) + ' '
          print(result)
  
    def reverse_graph(self):
      # Reset the variables
      new_adjacency_list = [[] for x in range(self.cnt_vertex)]
      new_dict_of_edges = dict()
      self.cnt_trees = 0
      self.list_of_trees = [Tree()]
      
      for node in self.list_of_nodes:
        node.color = 'white'
        node.started = 0
        node.finished = 0
        node.father = None
      
      for u in range(len(self.adjacency_list)):
        for v in self.adjacency_list[u]:
          new_adjacency_list[v].append(u)
          new_dict_of_edges[(v, u)] = None
      
      self.dict_of_edges = new_dict_of_edges
      self.adjacency_list = new_adjacency_list
      
      self.time = 0
      self.reverse = True

In [0]:
graph = Graph('toy_cycle.rud') 

In [0]:
graph.dfs_search()

In [364]:
graph.show_nodes()

Node: 0 Started at: 1 and Finished at: 10

Node: 1 Started at: 2 and Finished at: 3

Node: 2 Started at: 4 and Finished at: 9

Node: 3 Started at: 5 and Finished at: 6

Node: 4 Started at: 7 and Finished at: 8



In [365]:
print("Ordenação Topológica")
graph.show_topological()

Ordenação Topológica
Node0 --> Node2 --> Node4 --> Node3 --> Node1 --> 


In [366]:
graph.show_trees()

####################################################
0 1 2 3 4 


In [367]:
graph.list_type_of_edges()

0 1 Arvore
0 2 Arvore
0 3 Avanço
2 3 Arvore
3 0 Retorno
2 4 Arvore


In [0]:
graph.reverse_graph()

In [0]:
graph.dfs_search()

In [374]:
graph.show_trees()
print('Quantidade de Componentes Conexos: ' + str(len(graph.list_of_trees)))
print('Shapiro wilk')

####################################################
0 3 2 
4 
1 
Quantidade de Componentes Conexos: 3
